In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [13]:
cap = cv.VideoCapture('slow.flv')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 300,
                       qualityLevel = 0.2,
                       minDistance = 4,
                       blockSize = 5 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
print(p0)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
flag = True
while(flag):
    ret,frame = cap.read()
    if(frame is not None):
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
        img = cv.add(frame,mask)
        cv.imshow('frame',img)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
    else:
        flag = False
        break
cv.destroyAllWindows()
cap.release()

[[[323. 200.]]

 [[300. 104.]]

 [[237. 150.]]

 [[272. 104.]]

 [[330. 210.]]

 [[322. 210.]]

 [[309. 200.]]

 [[310. 211.]]

 [[264. 150.]]

 [[303. 204.]]

 [[175. 183.]]

 [[182. 185.]]

 [[329. 106.]]

 [[383. 211.]]

 [[349.  97.]]

 [[162. 191.]]

 [[268.  85.]]

 [[304.  86.]]

 [[395.  94.]]]
